In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version

spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"),sep=",",header=True)
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 1a. Split date column out into a year column
from pyspark.sql.functions import month, year
home_sales_df = home_sales_df.withColumn("year",year(home_sales_df['date']))
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [5]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('home_sales')

In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("SELECT year, ROUND(avg(price),2) AS average_price FROM home_sales WHERE bedrooms = 4 GROUP BY year ORDER BY year").show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("SELECT date_built,ROUND(avg(price),2) AS average_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built ORDER BY date_built").show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("SELECT date_built,ROUND(avg(price),2) AS average_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built ORDER BY date_built").show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT ROUND(avg(price),2) as average_price, view FROM home_sales GROUP BY view HAVING avg(price) >= 350000 ORDER BY view DESC").show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|average_price|view|
+-------------+----+
|   1061201.42|  99|
|   1053739.33|  98|
|   1129040.15|  97|
|   1017815.92|  96|
|    1054325.6|  95|
|    1033536.2|  94|
|   1026006.06|  93|
|    970402.55|  92|
|   1137372.73|  91|
|   1062654.16|  90|
|   1107839.15|  89|
|   1031719.35|  88|
|    1072285.2|  87|
|   1070444.25|  86|
|   1056336.74|  85|
|   1117233.13|  84|
|   1033965.93|  83|
|    1063498.0|  82|
|   1053472.79|  81|
|    991767.38|  80|
+-------------+----+
only showing top 20 rows

--- 0.722240686416626 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the query that filters out the view ratings with average price of greater than $350,000. 
# Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT ROUND(avg(price),2) as average_price, view FROM home_sales GROUP BY view HAVING avg(price) >= 350000 ORDER BY view DESC").show()


print("--- %s seconds ---" % (time.time() - start_time))


+-------------+----+
|average_price|view|
+-------------+----+
|   1061201.42|  99|
|   1053739.33|  98|
|   1129040.15|  97|
|   1017815.92|  96|
|    1054325.6|  95|
|    1033536.2|  94|
|   1026006.06|  93|
|    970402.55|  92|
|   1137372.73|  91|
|   1062654.16|  90|
|   1107839.15|  89|
|   1031719.35|  88|
|    1072285.2|  87|
|   1070444.25|  86|
|   1056336.74|  85|
|   1117233.13|  84|
|   1033965.93|  83|
|    1063498.0|  82|
|   1053472.79|  81|
|    991767.38|  80|
+-------------+----+
only showing top 20 rows

--- 0.4054889678955078 seconds ---


In [15]:
# 10. Partition the home sales dataset by the date_built field.
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [16]:
# 11. Read the formatted parquet data.
p_home_sales_df = spark.read.parquet("home_sales_partitioned")

In [17]:
# 12. Create a temporary table for the parquet data.
p_home_sales_df.createOrReplaceTempView("p_home_sales")

In [19]:
# 13. Run the query that filters out the view ratings with average price of greater than $350,000 with the parquet DataFrame. 
# Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

spark.sql("SELECT ROUND(avg(price),2) as average_price, view FROM p_home_sales GROUP BY view HAVING avg(price) >= 350000 ORDER BY view DESC").show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|average_price|view|
+-------------+----+
|   1061201.42|  99|
|   1053739.33|  98|
|   1129040.15|  97|
|   1017815.92|  96|
|    1054325.6|  95|
|    1033536.2|  94|
|   1026006.06|  93|
|    970402.55|  92|
|   1137372.73|  91|
|   1062654.16|  90|
|   1107839.15|  89|
|   1031719.35|  88|
|    1072285.2|  87|
|   1070444.25|  86|
|   1056336.74|  85|
|   1117233.13|  84|
|   1033965.93|  83|
|    1063498.0|  82|
|   1053472.79|  81|
|    991767.38|  80|
+-------------+----+
only showing top 20 rows

--- 0.706913948059082 seconds ---


In [20]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [21]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False